### 1. Retrieve live match participant's summoner_ids

In [1]:
from riotwatcher import LolWatcher, ApiError

from dotenv import dotenv_values
config = dotenv_values("../.env")

from sqlalchemy import create_engine
from sqlalchemy.exc import SQLAlchemyError
from sqlalchemy.engine import URL

engine = create_engine("mssql+pyodbc://DESKTOP-CBU1M8B/DataScience?driver=ODBC+Driver+17+for+SQL+Server")

import pandas as pd
from datetime import date, datetime
import time

""" initialize API """
watcher = LolWatcher(config['riotwatcher_api_key'])

var_region = 'euw1'


I will be using one random match, part of "featured_games" that are currently being played:

In [2]:
res_featured_games = watcher.spectator.featured_games(region=var_region)['gameList']

df_featured_games = pd.json_normalize(res_featured_games)

if df_featured_games.query('gameMode == "CLASSIC"').empty:
    print("Currently, there are no classic matches on featured games. Please try again later.")
    pass
else: ### right now, we are only picking one random game but it might be worthwhile to get more than one (all the classic, all the ranked, etc)
    var_game_id = df_featured_games.query('gameMode == "CLASSIC"').reset_index().loc[0, 'gameId']
    var_match_id = f"{var_region.upper()}_{var_game_id}"

    df_match = df_featured_games.query("gameId == @var_game_id").reset_index()


In [3]:
dict_featured_games = {
    "integration_datetime": [],
    "match_id": [],
}

while 1 > 0:
    time.sleep(10)
    var_integration_datetime = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S.%f')

    res_featured_games = watcher.spectator.featured_games(region=var_region)['gameList']
    df_featured_games = pd.json_normalize(res_featured_games)

    for var_match_id in df_featured_games['gameId']: ### .query('gameMode == "CLASSIC"')['gameId']:
        
        dict_featured_games['integration_datetime'].append(var_integration_datetime)

        dict_featured_games['match_id'].append(var_match_id)


In [8]:
tmp_df = pd.DataFrame.from_dict(dict_featured_games)
tmp_df.to_csv("feature_games_2022-12-04.csv")


In [62]:
var_integration_datetime = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S.%f')

dict_match_info = {
    "integration_datetime": [],
    "match_id": [],
    "match_queue": [],
    "game_mode": [],
    "game_type": [],
    "participant_id": [],
    "bot_boolean": [],
    "team_id": [],
    "summoner_name": [],
    "ban_champion_id": [],
    "pick_champion_id": [],
    "spell_1_id": [],
    "spell_2_id": [],
}

for var_participant_id in range(len(pd.json_normalize(df_match['participants']).columns)):

    """ integration_datetime """
    dict_match_info['integration_datetime'].append(var_integration_datetime)

    """ match_id """
    dict_match_info['match_id'].append(var_match_id)

    """ match_queue """
    dict_match_info['match_queue'] = df_match.loc[0,'gameQueueConfigId']

    """ game_mode """
    dict_match_info['game_mode'] = df_match.loc[0,'gameMode']

    """ game_type """
    dict_match_info['game_type'] = df_match.loc[0,'gameType']

    """ participant_id """
    dict_match_info['participant_id'].append(var_participant_id)

    """ bot_boolean """
    var_bot = pd.json_normalize(df_match['participants'])[var_participant_id][0]['bot']
        
    dict_match_info['bot_boolean'].append(var_bot)

    """ team_id """
    var_team_id = pd.json_normalize(df_match['participants'])[var_participant_id][0]['teamId']
        
    dict_match_info['team_id'].append(var_team_id)

    """ summoner_name """
    var_summoner_name = pd.json_normalize(df_match['participants'])[var_participant_id][0]['summonerName']
        
    dict_match_info['summoner_name'].append(var_summoner_name)

    """ ban_champion_id """
    var_ban_champion_id = pd.json_normalize(df_match['bannedChampions'])[var_participant_id][0]['championId']
        
    dict_match_info['ban_champion_id'].append(var_ban_champion_id)

    """ pick_champion_id """
    var_pick_champion_id = pd.json_normalize(df_match['participants'])[var_participant_id][0]['championId']
        
    dict_match_info['pick_champion_id'].append(var_pick_champion_id)

    """ spell_1_id """
    var_spell_1_id = pd.json_normalize(df_match['participants'])[var_participant_id][0]['spell1Id']
        
    dict_match_info['spell_1_id'].append(var_spell_1_id)

    """ spell_2_id """
    var_spell_2_id = pd.json_normalize(df_match['participants'])[var_participant_id][0]['spell2Id']
        
    dict_match_info['spell_2_id'].append(var_spell_2_id)

df_match_info = pd.DataFrame.from_dict(dict_match_info)


In [63]:
### do we need to keep bot_boolean? 
### team_id = 100 if blue or red?
### convert champion_id to champion_name using Data Dragon
### convert spell_id to spell_name using Data Dragon

# fetch winrate, banrate and pickrate from last 10 matches

df_match_info


,integration_datetime,match_id,match_queue,game_mode,game_type,participant_id,bot_boolean,team_id,summoner_name,ban_champion_id,pick_champion_id,spell_1_id,spell_2_id
0,2022-12-03 17:24:33.433765,EUW1_6136091387,420,CLASSIC,MATCHED_GAME,0,False,100,Dârth Vader,107,157,4,12
1,2022-12-03 17:24:33.433765,EUW1_6136091387,420,CLASSIC,MATCHED_GAME,1,False,100,Veignorem,120,555,4,14
2,2022-12-03 17:24:33.433765,EUW1_6136091387,420,CLASSIC,MATCHED_GAME,2,False,100,LagoDoom,38,134,4,12
3,2022-12-03 17:24:33.433765,EUW1_6136091387,420,CLASSIC,MATCHED_GAME,3,False,100,GPUQSXMH,238,33,4,11
4,2022-12-03 17:24:33.433765,EUW1_6136091387,420,CLASSIC,MATCHED_GAME,4,False,100,MRFCPBMMDD,-1,145,4,1
5,2022-12-03 17:24:33.433765,EUW1_6136091387,420,CLASSIC,MATCHED_GAME,5,False,200,GotzY,350,110,4,7
6,2022-12-03 17:24:33.433765,EUW1_6136091387,420,CLASSIC,MATCHED_GAME,6,False,200,ULTRAMENTAL69,120,121,11,4
7,2022-12-03 17:24:33.433765,EUW1_6136091387,420,CLASSIC,MATCHED_GAME,7,False,200,Targamas,39,74,4,14
8,2022-12-03 17:24:33.433765,EUW1_6136091387,420,CLASSIC,MATCHED_GAME,8,False,200,SERVE THE BEAST,267,142,4,14
9,2022-12-03 17:24:33.433765,EUW1_6136091387,420,CLASSIC,MATCHED_GAME,9,False,200,Desperate Nasus,876,10,4,14


In [64]:
var_integration_datetime = datetime.utcnow().strftime('%Y-%m-%d %H:%M:%S.%f')

dict_summoner_recent_stats = {
    "integration_datetime": [],
    "match_id": [],
    "frame_minute": [],
    "summoner_name": [],
    "current_gold": [],
    "total_gold": [],
}

for var_summoner_name in df_match_info['summoner_name']:

    print(var_summoner_name)

    dict_summoner_info = watcher.summoner.by_name(region=var_region, summoner_name=var_summoner_name)
    
    var_summoner_puuid = dict_summoner_info['puuid']
    var_summoner_level = dict_summoner_info['summonerLevel']
    
    list_matchlist = watcher.match.matchlist_by_puuid(region=var_region, puuid=var_summoner_puuid,count=10)
    time.sleep(10) 

    for var_match_id in list_matchlist:

        print(var_match_id)
        time.sleep(1) 

        dict_match_info = watcher.match.timeline_by_match(region=var_region,match_id=var_match_id)

        df_match_participants = pd.json_normalize(dict_match_info['info']['participants'])

        df_match_participant_id = df_match_participants.query('puuid == @var_summoner_puuid').reset_index()
        var_participant_id = str(df_match_participant_id.loc[0, 'participantId'])

        for var_frame_minute in range(len(dict_match_info['info']['frames'])):

            """ integration_datetime """
            dict_summoner_recent_stats['integration_datetime'].append(var_integration_datetime)

            """ match_id """
            dict_summoner_recent_stats['match_id'].append(var_match_id)

            """ summoner_name """
            dict_summoner_recent_stats['summoner_name'].append(var_summoner_name)

            """ frame_minute """
            dict_summoner_recent_stats['frame_minute'].append(var_frame_minute)

            """ current_gold """
            var_current_gold = dict_match_info['info']['frames'][var_frame_minute]['participantFrames'][var_participant_id]['currentGold']

            dict_summoner_recent_stats['current_gold'].append(var_current_gold)

            """ total_gold """
            var_total_gold = dict_match_info['info']['frames'][var_frame_minute]['participantFrames'][var_participant_id]['totalGold']

            dict_summoner_recent_stats['total_gold'].append(var_total_gold)


Dârth Vader
EUW1_6174518974
EUW1_6174457916
EUW1_6174408338
EUW1_6174382550
EUW1_6174311225
EUW1_6174267333
EUW1_6174222878
EUW1_6173833734
EUW1_6173652969
EUW1_6173567006
Veignorem
EUW1_6174518974
EUW1_6174114128
EUW1_6174094040
EUW1_6174066021
EUW1_6174040988
EUW1_6173725842
EUW1_6173289668
EUW1_6173040944
EUW1_6172979729
EUW1_6172941963
LagoDoom
EUW1_6174518974
EUW1_6173312701
EUW1_6171048087
EUW1_6170159456
EUW1_6168423734
EUW1_6168180407
EUW1_6163258481
EUW1_6163055399
EUW1_6161609277
EUW1_6161342327
GPUQSXMH
EUW1_6174518974
EUW1_6174482781
EUW1_6174419164
EUW1_6174386463
EUW1_6174367744
EUW1_6174315269
EUW1_6174246225
EUW1_6174192037
EUW1_6174144382
EUW1_6174116009
MRFCPBMMDD
EUW1_6174518974
EUW1_6174479899
EUW1_6173612401
EUW1_6173569289
EUW1_6173513696
EUW1_6173232470
EUW1_6173187496
EUW1_6172013378
EUW1_6171966523
EUW1_6171937862
GotzY
EUW1_6174518974
EUW1_6174479899
EUW1_6174419164
EUW1_6173962321
EUW1_6173954521
EUW1_6173931054
EUW1_6173907314
EUW1_6173874888
EUW1_6173844079

In [70]:
df_summoner_recent_stats = pd.DataFrame.from_dict(dict_summoner_recent_stats)

dict_total_gold_mean = {
    0: df_summoner_recent_stats.query('frame_minute == 0').groupby('summoner_name')['total_gold'].mean(),
    1: df_summoner_recent_stats.query('frame_minute == 1').groupby('summoner_name')['total_gold'].mean(),
    2: df_summoner_recent_stats.query('frame_minute == 2').groupby('summoner_name')['total_gold'].mean(),
    3: df_summoner_recent_stats.query('frame_minute == 3').groupby('summoner_name')['total_gold'].mean(),
    4: df_summoner_recent_stats.query('frame_minute == 4').groupby('summoner_name')['total_gold'].mean(),
    5: df_summoner_recent_stats.query('frame_minute == 5').groupby('summoner_name')['total_gold'].mean(),
    6: df_summoner_recent_stats.query('frame_minute == 6').groupby('summoner_name')['total_gold'].mean(),
    7: df_summoner_recent_stats.query('frame_minute == 7').groupby('summoner_name')['total_gold'].mean(),
    8: df_summoner_recent_stats.query('frame_minute == 8').groupby('summoner_name')['total_gold'].mean(),
    9: df_summoner_recent_stats.query('frame_minute == 9').groupby('summoner_name')['total_gold'].mean(),
    10: df_summoner_recent_stats.query('frame_minute == 10').groupby('summoner_name')['total_gold'].mean(),
    11: df_summoner_recent_stats.query('frame_minute == 11').groupby('summoner_name')['total_gold'].mean(),
    12: df_summoner_recent_stats.query('frame_minute == 12').groupby('summoner_name')['total_gold'].mean(),
    13: df_summoner_recent_stats.query('frame_minute == 13').groupby('summoner_name')['total_gold'].mean(),
    14: df_summoner_recent_stats.query('frame_minute == 14').groupby('summoner_name')['total_gold'].mean(),
    15: df_summoner_recent_stats.query('frame_minute == 15').groupby('summoner_name')['total_gold'].mean(),
}

df_total_gold_mean = pd.DataFrame.from_dict(dict_total_gold_mean)

df_total_gold_mean.sort_values(by=15, ascending=False)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
summoner_name,,,,,,,,,,,,,,,,
LagoDoom,1040.0,1044.3,1407.6,1833.000000,2421.000000,2974.400000,3609.400000,4165.300000,4626.900000,5097.700000,5734.700000,6227.000000,6879.000000,7319.400000,8072.100000,8645.000000
Dârth Vader,500.0,500.0,587.6,1113.400000,1449.200000,1801.700000,2329.700000,2711.600000,3202.600000,3802.100000,4282.200000,5048.000000,5508.800000,6066.800000,6649.900000,7421.500000
MRFCPBMMDD,500.0,500.0,626.2,933.400000,1347.000000,1751.400000,2132.500000,2604.700000,3042.200000,3458.000000,4038.200000,4557.700000,4991.100000,5483.100000,6052.000000,6504.700000
GPUQSXMH,500.0,500.0,622.7,1071.777778,1520.111111,2042.333333,2362.333333,2718.333333,3193.888889,3568.000000,3918.888889,4488.000000,5035.777778,5368.333333,5917.888889,6380.333333
ULTRAMENTAL69,500.0,500.5,625.5,1048.500000,1339.900000,1718.400000,2125.100000,2477.300000,2877.900000,3336.700000,3762.900000,4211.900000,4742.300000,5143.000000,5685.200000,6146.500000
GotzY,500.0,501.0,555.8,1019.300000,1332.100000,1760.200000,2137.300000,2495.700000,2986.500000,3363.300000,3836.100000,4233.500000,4754.400000,5277.300000,5773.100000,6143.500000
Desperate Nasus,500.0,500.0,556.5,849.333333,1286.222222,1680.333333,2052.444444,2356.222222,2740.666667,3148.222222,3702.111111,4279.222222,4686.777778,5002.222222,5508.444444,6089.555556
SERVE THE BEAST,500.0,500.0,581.2,934.800000,1197.400000,1584.800000,2042.000000,2403.000000,2910.400000,3395.900000,3774.300000,4203.200000,4682.600000,5032.900000,5570.500000,6053.600000
Veignorem,590.0,610.4,723.0,1009.100000,1366.000000,1715.800000,2047.300000,2378.000000,2686.300000,2936.200000,3352.300000,3699.800000,4048.300000,4366.000000,4681.100000,5064.600000
